# Low Rank Sparse Noise Decomposition - GoDec

## Description

This is the Python 3.7 traduction of the AISTATS 2013 GreBsmo code (Zhou and Tao) available here in its Matlab version :
https://tianyizhou.wordpress.com/2014/05/23/aistats-2013-grebsmo-code-is-released/

More details about the code :
https://nuit-blanche.blogspot.com/2014/07/grebsmo-greedy-bilateral-sketch.html

Our Python version can take 8bit or 16bit TIFF stacks as inputs, and more common .avi movies.
The RGB .avi are converted in grayscale.

## File description

- The Samples folder contains two test files : a .avi movie (Escalator.avi) and a b/w 8bit TIFF stack of the same movie (Escalator-1000f-8b.tif). For space reasons, the TIFF stack contains only the first 1000 frames of the movie.

## Output
The output will be 2 TIFF stacks :

- 2-Background.tif : the Low Rank component TIFF stack, corresponding to the background

- 3-Sparse.tif : the Sparse component TIFF file, containing the moving objects of interest


The function takes the following arguments :

- Namefile : path of the .avi movie or TIFF stack
- rank : rank(L)<=rank
- tau : soft thresholding
- tol : error tolerance
- power: >=0, power scheme modification, increasing it lead to better accuracy and more time cost
- k : rank stepsize
- dynamicrange : grayscale depth (8 or 16) of the output TIFF files
- length : the number of frames on which to run the routine.

rank, tau, power, tol and k are parameters coming from the code of Zhou and Tao.

## REFERENCE

Tianyi Zhou and Dacheng Tao, "GoDec: Randomized Lo-rank & Sparse Matrix Decomposition in Noisy Case", ICML 2011

Tianyi Zhou and Dacheng Tao, "Greedy Bilateral Sketch, Completion and  Smoothing", AISTATS 2013.

Python Implementation: J. Fattaccioli (Department of Chemistry, ENS)

Date: November 2021

# Initial Setup

In [1]:
#conda activate ImageAnalysis
#Environment to activate on Jacques's machine

%gui qt

import napari
import sys
import numpy as np
import pywt
import scipy as sc
import scipy.sparse.linalg as linalg
import cv2 as cv
import session_info

from skimage import data, io, transform
from PyQt5.QtWidgets import QApplication, QWidget, QInputDialog, QLineEdit, QFileDialog
from PyQt5.QtGui import QIcon

def gui_fname(dir=None):
    """Select a file via a dialog and return the file name."""
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="TIFF (*.tif) ;; AVI (*.avi)")
    
    filename = fname[0].split('/')[-1]
    path=fname[0].replace(filename, '')
    
    return (path, filename, fname[1])

def import_image(filename: str, max_frames: int) -> np.ndarray:
    """
    Retrieve an tiff image stack in a numpy array
    max_frames : maximum number of frames. 0 by default: all frames are loaded
    """

    NameFileParsed = filename.split(".")
    print(NameFileParsed[-1])

    if NameFileParsed[-1] == ("tif" or "tiff"):
        Image = io.imread(filename)
        Image.astype(float)

        if max_frames > Image.shape[0]:
            max_frames = Image.shape[0]

        if max_frames != 0:
            Image = Image[
                :max_frames,
            ]
        print("TIF stack loading OK")

    elif NameFileParsed[-1] == "avi":
        Cap = cv.VideoCapture(filename)
        frame_count = int(cv.VideoCapture.get(Cap, int(cv.CAP_PROP_FRAME_COUNT)))

        if max_frames > frame_count:
            max_frames = frame_count

        if max_frames != 0:
            Frames = max_frames

        Width = int(cv.VideoCapture.get(Cap, int(cv.CAP_PROP_FRAME_WIDTH)))
        Height = int(cv.VideoCapture.get(Cap, int(cv.CAP_PROP_FRAME_HEIGHT)))
        Temp = np.zeros((Frames, Height, Width))

        for framenumber in range(Frames):
            if (framenumber + 1) % 10 == 0:
                print("Frame number = ", framenumber + 1)

            ret, frame = Cap.read()
            # if frame is read correctly ret is True
            if not ret:
                print("Can't receive frame (stream end?). Exiting...")
                break
            Temp[framenumber::] = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        Cap.release()
        Image = Temp
        print("AVI loading OK")
    else:
        raise RuntimeError("The file extension should be .tif, .tiff or .avi.")

    NImage = Image.shape[0]
    HImage = Image.shape[1]
    WImage = Image.shape[2]
    return Image, NImage, HImage, WImage

def vectorize(Image, NImage, HImage, WImage) -> np.ndarray:
    """
    nparray(N, H, W)*int*int*int -> np.array(N,H*W)
    Convert the image stack (3d) in 2D
    This function is necessary to perform the GoDec process
    """

    FinalImage = np.zeros((NImage, HImage * WImage))
    for i in range(NImage):
        FinalImage[i, :] = np.reshape(Image[i, :, :], HImage * WImage)

    return FinalImage

def reconstruct(vector, line, col, time) -> np.ndarray:
    """
    np.array(N,H*W)*H(int)*W(int)*N(int)*str*int -> nparray(N, H, W)
    Reconstruct the image stacks after the GoDecprocess
    Save the nparray in a tif stack 'path' with the given dynamic range ( 8 or 16)
    """
    image = np.zeros((time, line, col))
    for i in range(time - 1):
        temp = vector[i, :]
        image[i, :, :] = temp.reshape((line, col))

    return image

def save(data, path, dynamicrange=8) -> None:
    """
    nparray*str*int -> *
    This function save the nparray data as tif file with a given dynamic range
    Dynamic range = 8 or 16
    """

    # Conversion of the pixel values in uint8
    data = ((data - np.amin(data)) / np.amax(data - np.amin(data))) * (
        2 ** dynamicrange - 1
    )
    if dynamicrange == 8:
        data = data.astype(np.uint8)
    elif dynamicrange == 16:
        data = data.astype(np.uint16)
    else:
        raise Exception("The dynamic range should be equal to 8 or 16 (bits)")
    
    print(path)
#    Saving tif file
    io.imsave(path, data)
    
    
def GreGoDec(D, rank, tau, tol, power, k):
    """
    INPUTS:
    D: nxp data matrix with n samples and p features
    rank: rank(L)<=rank
    tau: soft thresholding
    power: must be positive int. Power scheme modification, increasing it leads to better accuracy at the cost of time
    k: rank stepsize

    OUTPUTS:
    D : Input data matrix
    L:Low-rank part
    S:Sparse part

    The code uses these functions :
    https://pywavelets.readthedocs.io/en/latest/ref/thresholding-functions.html
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html
    """
    # Transposition in case the array has a wrong shape

    if D.shape[0] < D.shape[1]:
        D = np.transpose(D)

    normD = np.linalg.norm(D)
    # initialization of L and S
    rankk = round(rank / k)
    error = np.zeros((rank * power, 1), dtype=float)

    # Computation of the singular values
    X, s, Y = linalg.svds(D, k)

    s = s * np.identity(k)
    X = X.dot(s)
    L = X.dot(Y)

    # Wavelet thresholding functions
    # https://pywavelets.readthedocs.io/en/latest/ref/thresholding-functions.html
    S = pywt.threshold(D - L, tau, 'soft')

    # Calcul de l'erreur
    T = D - L - S
    error[0] = np.linalg.norm(T) / normD

    iii = 1
    stop = False
    alf = 0
    for r in range(1, rankk + 1):
        print("Step ", r, " over", rankk) #Etat d'avancement dans la boucle
        r = r - 1
        # parameters for alf
        rrank = rank
        est_rank = 1  # Est_rank est toujours = à 1 (?) ?
        alf = 0
        increment = 1

        if iii == power * (r - 2) + 1:
            iii = iii + power
        for iter in range(1, power + 1):
            print(100*iter/power, "%")#Etat d'avancement dans la boucle
            # Update of X
            X = abs(L.dot(np.transpose(Y)))

            # QR decomposition of X=L*Y' matrix
            X, R = np.linalg.qr(X)

            # Update of Y
            Y = np.transpose(X).dot(L)
            L = X.dot(Y)
            # Update of S
            T = D - L
            S = pywt.threshold(T, tau, mode='soft')

            # Error, stopping criteria
            T = T - S
            ii = iii + iter - 1

            error[ii] = np.linalg.norm(T) / np.linalg.norm(D)

            if error[ii] < tol:
                stop = True
                break

            if rrank != rank:
                rank = rrank
                if est_rank == 0:
                    alf = 0
                    continue

            # Adjust alf
            ratio = error[ii] / error[ii - 1]
            # Interzmediate variables
            X1 = X
            Y1 = Y
            L1 = L
            S1 = S
            T1 = T

            if ratio >= 1.1:
                increment = max(0.1 * alf, 0.1 * increment)
                X = X1
                Y = Y1
                L = L1
                S = S1
                T = T1
                error[ii] = error[ii - 1]
                alf = 0

            elif ratio > 0.7:
                increment = max(increment, 0.25 * alf)
                alf = alf + increment

            # Update of L
            X1 = X
            Y1 = Y
            L1 = L
            S1 = S
            T1 = T
            L = L + (1 + alf) * T

            # Add corest
            if iter > 8:
                if np.mean(np.divide(error[ii - 7 : ii], error[ii - 8])) > 0.92:
                    iii = ii

                    if Y.shape[1] - X.shape[0] >= k:
                        Y = Y[0 : X.shape[0] - 1, :]
                    break

        # Stop
        if stop == True:
            break

        # Coreset
        if r + 1 < rankk:
            RR = np.random.randn(k, D.shape[0])
            v = RR.dot(L)
            Y = np.block([[Y], [v]])

    # error[error==0]=[]
    error = [error != 0]
    L = X.dot(Y)

    if D.shape[0] > D.shape[1]:
        L = np.transpose(L)
        S = np.transpose(S)

    return np.transpose(D), L, S

def DecomposeGoDec(FinalImage, rank=3, power=5, tau=7, tol=0.001, k=2,  max_frames=0):
    """
    The entry point function to call to do the decomposition

    StartingImage: np.array (W, H, N)
    rank: ralk(L) <= rank
    power: must be positive int. Power scheme modification, increasing it leads to better accuracy at the cost of time
    tau: soft thresholding
    tol: error tolerance
    k: rank stepsizes
    dynamicrange: grayscale depth (8 or 16) of the output TIFF files
    max_frames: the number of frames on which to run the routine (default is 0 which means all the frames)
    """

    D, L, S = GreGoDec(FinalImage, rank, tau, tol, power, k)
    print("GoDec OK")

    return (D, L, S)

# Start of the procedure

## Retrieve the file name and path

In [2]:
#Retrieve the file name and path using the QtGUI open file dialog

path, filename, filetype = gui_fname()
print(path, filename, filetype)

#noise, bgd, sparse = GreGoDec.DecomposeGoDec(path+filename, 100)

/Users/jacques/Dropbox/Codes/Python/ImageAnalysis/GoDec/Samples/ 1-Original.tif TIFF (*.tif)


## Number of frames to analyse

0 is the value per default : all the frames will be imported
frame_number should be smaller than the actual number of frames in the tiff_stack or movie

In [3]:
max_frames = 0 #If max_frame = 0 : import all the slices of the stack

## Load the tiff stack or the avi file

In [4]:
stack, frames, height, width = import_image(path + filename, max_frames)
print("width =", width, "height =", height, "frames =", frames)

tif
TIF stack loading OK
width = 160 height = 130 frames = 100


In [5]:
stack.shape
stack = stack[::4,0:,0:]
frames = stack.shape[0]

## Transform the stack in a (width x length, frames) array

In [6]:
vector_stack = np.zeros((frames, width * height), dtype=int)
vector_stack = vectorize(stack, frames, height, width)
print("Image Vectorization OK")

Image Vectorization OK


## Apply GoDec to the stack

- rank: ralk(L) <= rank
- power: must be positive int. Power scheme modification, increasing it leads to better accuracy at the cost of time
- tau: soft thresholding
- tol: error tolerance
- k: rank stepsizes

In [10]:
rank=3
power=5 
tau=7
tol=0.001
k=2

In [11]:
D, L, S = DecomposeGoDec(vector_stack, rank, power, tau, tol, k)

D = reconstruct(D, height, width, frames)
print("Reconstruction Original Stack OK")
L = reconstruct(L, height, width, frames)
print("Reconstruction Low-rank OK")
S = reconstruct(S, height, width, frames)
print("Reconstruction Sparse OK")
G = reconstruct(D - L - S,height, width, frames)
print("Reconstruction Noise OK")
print("Full Process OK")

#D, L, S = GreGoDec(vector_stack, rank, tau, tol, power, k)



Step  1  over 2
20.0 %
40.0 %
60.0 %
80.0 %
100.0 %
Step  2  over 2
20.0 %
40.0 %
60.0 %
80.0 %
100.0 %
GoDec OK
Reconstruction Original Stack OK
Reconstruction Low-rank OK
Reconstruction Sparse OK
Reconstruction Noise OK
Full Process OK


In [12]:
if 'viewer' in locals(): # myVar exists.
    viewer.close()
    
viewer = napari.view_image(stack, name="Original data")
print(viewer)
viewer.add_image(S, name="Sparse array : moving objects")
viewer.add_image(G, name="Noise array")
viewer.add_image(L, name="Low-rank array : background")

napari.Viewer: napari


<Image layer 'Low-rank array : background' at 0x7fb44a53ee50>

# Saving images

If you are satisfied with the decomposition, please execute the following cell. If you are not satisfied, please adapt the rank/power/tau parameters accordingly

In [14]:
dynamic_range = 8 #(or 16). 8bits is by default, if the parameter is left empty

name = path + filename.split(".")[-2] + "_sparse.tif"
save(S, name, dynamic_range)
name = path + filename.split(".")[-2] + "_background.tif"
save(L, name, dynamic_range)
name = path + filename.split(".")[-2] + "_noise.tif"
save(G, name, dynamic_range)

/Users/jacques/Dropbox/Codes/Python/ImageAnalysis/GoDec/Samples/Escalator-1000f-8b_sparse.tif
/Users/jacques/Dropbox/Codes/Python/ImageAnalysis/GoDec/Samples/Escalator-1000f-8b_background.tif
/Users/jacques/Dropbox/Codes/Python/ImageAnalysis/GoDec/Samples/Escalator-1000f-8b_noise.tif


In [29]:
print(viewer)

napari.Viewer: napari


In [16]:
session_info.show()